In [1]:
using VCFTool

In [2]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_example_file_path = joinpath(input_dir, "example.vcf.gz")

"../input/example.vcf.gz"

In [3]:
vcf_738 = read_vcf(vcf_738_file_path)

vcf_example = read_vcf(vcf_example_file_path)

vcf_to_use = vcf_738

,CHROM,POS,ID,REF,ALT,QUAL,FILTER
,String,Int64,String,String,String,String,String
1,chr1,10439,.,AC,A,72,PASS
2,chr1,13284,.,G,A,60,LowGQX;NoPassedVariantGTs
3,chr1,13868,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
4,chr1,15274,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
5,chr1,15903,.,G,GC,209,PASS
6,chr1,16683,.,T,G,0,LowGQX;NoPassedVariantGTs
7,chr1,16713,.,T,G,0,LowGQX;LowDepth;NoPassedVariantGTs
8,chr1,16731,.,C,G,0,LowGQX;NoPassedVariantGTs
9,chr1,16734,.,T,G,4,LowGQX;NoPassedVariantGTs


## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

## Julia DataFrame Query 

In [ ]:
regions = Dict([
        ("chr2", 240630710),
        ("chr7", 150999023),
        ("chr10", 99836239),
        ("chr10", 99851537),
        ("chr16", 88646828),
        ("chr19", 51354484),
        ("chr21", 36146408),
        ])

query_regions(
    regions, 
    vcf_to_use,
    output_dir,
)

## JuliaDB

In [4]:
using JuliaDB

In [5]:
t = table(rand(Bool, 10), rand(10), pkey=1)

Table with 10 rows, 2 columns:
1      2
────────────────
false  0.837107
false  0.694487
false  0.880145
true   0.16975
true   0.150182
true   0.337319
true   0.0063954
true   0.94541
true   0.345
true   0.939661

In [ ]:
using DataFrames

using CSV: File

io = open(vcf_738_file_path)

vcf = DataFrame(File(io; delim='\t', header=false))

print(vcf)

close(io)

In [ ]:
vcf_example = loadtable(vcf_738_file_path, skiplines_begin="#")